# Cleaning the Data

## Get information about the data

In [1]:
#import pandas for use in cleaning data and print number of columns and rows
import pandas as pd

input_file = 'WC_Claims_Beginning_2000.csv'
filtered_output_file = 'WC_Claims_filtered.csv'
final_output_file = 'WC_Claims_2020-2022.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(input_file)

# Get the number of rows and columns
num_rows, num_columns = df.shape

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 4843711
Number of Columns: 54


In [2]:
# Show the name of all headers

first_row = df.iloc[0]

print(first_row)

Accident                                                       Y
Accident Date                                         10/02/2000
Age at Injury                                                 32
Alternative Dispute Resolution                                 Y
ANCR Date                                                    NaN
Assembly Date                                         10/13/2000
Attorney/Representative                                        N
Average Weekly Wage                                          0.0
Birth Year                                                  1968
C-2 Date                                                     NaN
C-3 Date                                                     NaN
Carrier Name                                ELECTRICAL EMPLOYERS
Carrier Type                                    4A. SELF PRIVATE
Claim Identifier                                            5031
Claim Injury Type                                    2. NON-COMP
Claim Type               

## Begin Cleaning

### Remove all records with accident date prior to 2020

In [3]:
import csv
from datetime import datetime

def is_date_in_range(date_str, start_year, end_year):
    try:
        date = datetime.strptime(date_str, "%m/%d/%Y")
        year = date.year
        return start_year <= year <= end_year
    except ValueError:
        return False

filtered_rows = []

with open(input_file, 'r', newline='') as infile:
    reader = csv.reader(infile)
    header = next(reader)

    for row in reader:
        date_column_index = 1  # This is the Accident Date column
        accident_date = row[date_column_index]

        if is_date_in_range(accident_date, 2020, 2022):
            filtered_rows.append(row)

with open(filtered_output_file, 'w', newline='') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(header)
    writer.writerows(filtered_rows)

### Remove unneeded columns and rows and shorten some column names

In [4]:
df = pd.read_csv(filtered_output_file, low_memory=False)

# Keep only the specified columns
df = df[['Accident', 'Accident Date', 'Age at Injury', 'Attorney/Representative', 'Claim Identifier', 'Claim Injury Type',
        'Claim Type', 'Closed Count', 'Current Claim Status', 'Gender', 'Highest Process', 'Industry Code Description',
        'WCIO Cause of Injury Description', 'WCIO Nature of Injury Description', 'WCIO Part Of Body Description']]

# Only include rows where 'Gender' is not 'X' or 'U'
df = df[(df['Gender'] != 'X') & (df['Gender'] != 'U')]

# Rename long or awkward column names
df.rename(columns={'Attorney/Representative': 'Represented'}, inplace=True)
df.rename(columns={'WCIO Cause of Injury Description': 'Cause of Injury'}, inplace=True)
df.rename(columns={'WCIO Nature of Injury Description': 'Nature of Injury'}, inplace=True)
df.rename(columns={'WCIO Part Of Body Description': 'Part of Body'}, inplace=True)

## Change categorical columns to numerical for ease of processing

Because having a hearing is all the same, whether it is for the judge, an appeal, or a settlement, all of the HEARING categories are set to a 4 and will be grouped together.

Similarly, parts of the body are grouped together as 1. Arms/Hands, 2. Legs/Feet, 3. Mouth/Face, 4. Neck/Spine, 5. Heart/Internal Organs, 6. Hips/Pelvis, 7. Throat/Lungs, 8. Brain/Head, 9. Multiple Parts of Body, 10. No Physical Injury or Insufficient Data to Classify, and 11. Artificial Appliance.

In [5]:
# Convert 'Highest Process' column to numbers for easier processing
df['Highest Process'] = df['Highest Process'].replace({'1.  NO RESOLUTIONS': 1, '2. ADMINISTRATIVE DETERMINATION': 2,
                                                       '3B. CONCILIATION - MEETING': 3,'4A. HEARING - JUDGE': 4,
                                                       '4B. HEARING - APPEAL': 4,'4C. HEARING - SETTLEMENT': 4})

# Merge 'Part of Body' to one area and convert 'Part of Body' column to numbers for easier processing
df['Part of Body'] = df['Part of Body'].replace({'HAND': 1, 'FINGER(S)': 1, 'UPPER ARM': 1, 'SHOULDER(S)': 1,'ELBOW': 1,
                                                'LOWER ARM': 1, 'WRIST': 1, 'WRIST (S) & HAND(S)': 1, 'THUMB': 1,'KNEE': 2,
                                                'ANKLE': 2, 'LOWER LEG': 2, 'FOOT': 2, 'TOES': 2, 'UPPER LEG': 2,
                                                'GREAT TOE': 2, 'EYE(S)': 3, 'TEETH': 3, 'FACIAL BONES': 3, 'NOSE': 3,
                                                'EAR(S)': 3, 'MOUTH': 3, 'LOWER BACK AREA': 4, 'MULTIPLE NECK INJURY': 4,
                                                'SPINAL CORD': 4, 'LUMBAR & OR SACRAL VERTEBRAE (VERTEBRA': 4,
                                                'UPPER BACK AREA': 4, 'DISC': 4, 'VERTEBRAE': 4, 'SOFT TISSUE': 5, 'LUNGS': 7,
                                                'INTERNAL ORGANS': 5, 'HEART': 5, 'HIP': 6, 'BUTTOCKS': 6, 'PELVIS': 6,
                                                'SACRUM AND COCCYX': 6, 'LARYNX': 7, 'TRACHEA': 7, 'SKULL': 8, 'BRAIN': 8,
                                                'MULTIPLE HEAD INJURY': 8, 'MULTIPLE UPPER EXTREMITIES': 1, 'MULTIPLE': 9,
                                                'MULTIPLE LOWER EXTREMITIES': 2, 'MULTIPLE TRUNK': 9, 'NO PHYSICAL INJURY': 10,
                                                'INSUFFICIENT INFO TO PROPERLY IDENTIFY - UNCLASSIFIED': 10, 
                                                'BODY SYSTEMS AND MULTIPLE BODY SYSTEMS': 9, 'ABDOMEN INCLUDING GROIN': 5,
                                                'CHEST': 7, 'WHOLE BODY': 9, 'MULTIPLE BODY PARTS (INCLUDING BODY': 9,
                                                'ARTIFICIAL APPLIANCE': 11})

# Replace NaN with 10 for No Injury or No data
df['Part of Body'] = df['Part of Body'].fillna(10)

# Convert 'Represented' column to numbers for easier processing
df['Represented'] = df['Represented'].replace({'Y': 1, 'N': 0})

# Convert 'Gender' column to numbers for easier processing
df['Gender'] = df['Gender'].replace({'F': 1, 'M': 0})

# Write the modified DataFrame to a new CSV file
df.to_csv(final_output_file, index=False)

## Verify record and column counts for clean data

In [6]:
df = pd.read_csv(final_output_file)

# Get the number of rows and columns
num_rows, num_columns = df.shape

print(f"Number of Rows: {num_rows}")
print(f"Number of Columns: {num_columns}")

Number of Rows: 746216
Number of Columns: 15


In [7]:
# List all unique values in the 'Claim Injury Type' column
unique_values = df['Claim Injury Type'].unique()

# Print the unique values
print("Unique values in 'Claim Injury Type' column:")
print(unique_values)

Unique values in 'Claim Injury Type' column:
['1. CANCELLED' '2. NON-COMP' '4. TEMPORARY' '3. MED ONLY'
 '5. PPD SCH LOSS' '6. PPD NSL' '8. DEATH' '7. PTD']


In [8]:
import pandas as pd
final_output_file = 'WC_Claims_2020-2022.csv'
df = pd.read_csv(final_output_file)
print(df.dtypes)

Accident                      object
Accident Date                 object
Age at Injury                  int64
Represented                    int64
Claim Identifier               int64
Claim Injury Type             object
Claim Type                    object
Closed Count                   int64
Current Claim Status          object
Gender                         int64
Highest Process                int64
Industry Code Description     object
Cause of Injury               object
Nature of Injury              object
Part of Body                 float64
dtype: object


In [9]:
first_row = df.iloc[0]

print(first_row)

Accident                                              Y
Accident Date                                06/01/2021
Age at Injury                                        69
Represented                                           1
Claim Identifier                                5798945
Claim Injury Type                          1. CANCELLED
Claim Type                   WORKERS COMPENSATION CLAIM
Closed Count                                          3
Current Claim Status                     CASE CANCELLED
Gender                                                0
Highest Process                                       2
Industry Code Description          EDUCATIONAL SERVICES
Cause of Injury                       REPETITIVE MOTION
Nature of Injury                 CARPAL TUNNEL SYNDROME
Part of Body                                        1.0
Name: 0, dtype: object


In [10]:
final_output_file = 'WC_Claims_2020-2022.csv'
df = pd.read_csv(final_output_file)

# List all unique values in the 'Part of Body' column
unique_values = df['Part of Body'].unique()

# Print the unique values
print("Unique values in 'Part of Body' column:")
print(unique_values)

Unique values in 'Part of Body' column:
[ 1.  2.  3.  4.  5.  9.  8.  6.  7. 10. 11.]
